In [9]:
#import numpy for array processing
import numpy as np #np is shorten, numpy is to inicate that the data will be in matrices
#
import os #use to load and pre process the dataset
import time

#import tensorflow-related modules
#from tensorflow.keras.applications.vgg16 import VGG16
#from tensorflow.keras.applications.vgg19 import VGG19
#from tensorflow.keras.applications import DenseNet121
#from tensorflow.keras.applications import DenseNet169
#from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications import MobileNetV2


from tensorflow.keras.preprocessing import image 
# it provides utilities for working wt imgae data, text data and sequence data
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions #
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import utils
#import scikit-learn modules
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt #library for plotting graph

from tensorflow import keras
from tensorflow.keras import layers

In [10]:
#### This cell is about loading and going through all the images
#get the current directory
PATH = os.getcwd()
data_path = f"{PATH}/picture"
print(data_path)

#get the files inside the directory
data_dir_list = os.listdir(data_path)
print(data_dir_list)

img_data_list=[] #Create array to store image

#loop through the directory
for dataset in data_dir_list: #read data in files
    dir_ff = f"{data_path}/{dataset}"
    img_list=os.listdir(dir_ff)#+'/'+ dataset) #explore images in data_path and return a list containing name entries to direcory
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:  #for image in img_list
        img_path = f"{data_path}/{dataset}/{img}" #: set img as img_path
        img = image.load_img(img_path, target_size=(224, 224)) #Load image aordding to the size given in target
        x = image.img_to_array(img)#take the variable img in array
        #x = np.expand_dims(x, axis=0)
        x = preprocess_input(x) #for predefined imagenet networks: it's a function meant to adequate image accord to what model require
        #print('Input image shape:', x.shape)
        img_data_list.append(x) #add x (in this case an image) to the list (in this case array) 
print(len(img_data_list))

img_data = np.array(img_data_list);
print(img_data.shape);


C:\Users\tnich\Documents\projectnn/picture
['dogs', 'Humans']
Loaded the images of dataset-dogs

Loaded the images of dataset-Humans

404
(404, 224, 224, 3)


In [3]:
#img_data = np.rollaxis(img_data,1,0); #what is this for?? 
# print(img_data.shape);
# imgdata = img_data[0]; # what's for?? 
# print(img_data.shape)

In [11]:
#define classes
num_class = 2;
num_of_sample = img_data.shape[0];
print('Number of samples:',num_of_sample);

#label images
labels = np.ones((num_of_sample), dtype='int64');
labels[0:202]=0 ;
labels[202:404]=1;
names = ['Dog', 'Human'];

#assign Y as label
y = utils.to_categorical(labels, num_class);
#random dataset for training
x,y = shuffle(img_data, y, random_state=2);

Number of samples: 404


In [5]:
# Split the dataset
#split the array or matrices into random tran and test subset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

In [12]:
## custom vgg model
# this for training the classifier 
img_input = Input(shape=(224,224,3));

model = MobileNetV2(input_tensor = img_input, include_top = True, weights = 'imagenet');

#code model VGG19 from https://github.com/keras-team/keras-applications/blob/master/keras_applications/vgg19.py
#model = VGG19(include_top=True, weights='imagenet', input_tensor= img_input)

#model = keras.applications.densenet.DenseNet121(include_top=True, weights='imagenet', input_tensor= img_input)

#code model MobileNetV2  from https://github.com/keras-team/keras-applications/blob/master/keras_applications/mobilenet_v2.py
#model = keras.applications.MobileNetV2(include_top=True, weights='imagenet', input_tensor= img_input);

#summary part
model.summary()

#weights
#output from VGG16 will become the last layer
#last_layer = model.get_layer('fc2').output #for VGG16 $ VGG19
last_layer = model.get_layer('Logits').output #for MobileNetV2
#last_layer = model.get_layer('fc1000').output #for DenseNet

#connect last layer with the hiddenlayers

#this line is same for VGG16 md VGG19 and MobileNet
out = Dense(num_class, activation = 'softmax', name = 'output')(last_layer);
#

custom_model = Model(img_input, out);
custom_model.summary();

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [13]:
#only trainging the last layer
for layer in custom_model.layers[:-1]:
    layer.trainable = False

custom_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

t=time.time()
#t = now()
hist = custom_model.fit(X_train, y_train, batch_size=32, epochs=4, verbose=1, validation_data=(X_test, y_test))
print('Training time: %s' % (t - time.time()))
(loss, accuracy) = custom_model.evaluate(X_test, y_test, batch_size=10, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))


Train on 323 samples, validate on 81 samples
Epoch 1/4
 32/323 [=>............................] - ETA: 1:04

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node model_1/Conv1/Conv2D (defined at C:\Users\tnich\Anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]] [Op:__inference_distributed_function_26476]

Function call stack:
distributed_function


In [ ]:
#%%

# visualizing losses and accuracy
train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['accuracy']
val_acc=hist.history['val_accuracy']
xc=range(4)

plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

In [ ]:
# Save predictions for future checks
predictions = custom_model.predict(X_test)

#result = predictions[0]


In [ ]:
print('Predicted:', predictions)

In [ ]:
# Save the model
custom_model.save('MobileNetV2_my_model2.h5')
